## 字符串匹配算法

In [8]:
def BF(text, pattern):
    n, m = len(text), len(pattern)
    for i in range(n - m + 1):
        for j in range(m):
            if text[i + j] != pattern[j]:
                break
        else:
            return i
    return -1

- 对模式串求前缀和后缀相等的最大长度

例如：模式串```abbabb```

| length | 1 | 2 | 3 | 4 | 5 | 6 |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| prefix | a | ab | abb | abba | abbab | \\ |
| suffix | b | bb | abb | babb | bbabb | \\ |

故字符串 ```abbabb``` 的最大长度为 3

求模式串的 $next$ 数组

例如：模式串```aabaabsaabaabst```

```i```位置的信息只和之前的字符串有关，由于 ```0```号位置之前没有字符串所以 $next[0]=-1$

```1```号位置，由于前缀和后缀都不能取到整体，所以$next[1]=0$

第```i```号位置 $next[i]=pattern[0]...pattern[i-1]$的前缀和后缀相等的最大长度


- text 和 pattern 的匹配过程

例如：

text 是 ```abbstkscabbstks...```

| index | 0 | 1| 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | ... |
| :--: | :--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |
| character | a | b | b | s | t | k | s | c | a | b | b | s | t | k | s | ... |

pattern 是```abbstkscabbstkz...```

| index | 0 | 1| 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | ... |
| :--: | :--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |:--: |
| character | a | b | b | s | t | k | s | c | a | b | b | s | t | k | z | ... |

由于 $text[14] != pattern[14]$ 又 $text[8]...text[13]=pattern[0]...pattern[5]$ 所以下次直接比较 $text[14]$ 和 $pattern[6]$ 即可

证明**跳过部分无论如何都匹配不到 pattern**

**text**

| index | ... | i | ...| k | ... | j | ... | x |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| character | ... | a | ... | . | . |. |... | . |


**pattern**

| index | 0 | ... | p |  ...| z | ... | q | ... | y |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| character | a | ...| . | ... | . | ... | . | ... | . |

$text$ 的子串 $text[i]...text[x-1]$和 $pattern[0]...pattern[y-1]$ 是匹配的只有 $text[x]$ 和 $pattern[y]$ 是不匹配的 

设 $pattern[0]...pattern[p]$ 和 $pattern[z]...pattern[q]$ 是 $pattern[y]$ 的最长的前后缀

假设位置 $i$ 到位置$j$ 中间的位置 $k$ 开头的子串能匹配到 $pattern$ 所以有 $text[k]...text[x]$ 和 $pattern$有等量的前缀又由于 $text[j]...text[x-1]$ 和 $pattern[0]...pattern[p]$ 是相等的，所以有 $x-1-j=p$ 而假设 $x-1-k>p$ 故假设不成立

In [35]:
def KMP(text: str, pattern: str):
    def construct(pattern, m):
        if m == 1:
            return [-1]
        nxt = [0 for i in range(m)]
        nxt[0] = -1
        i, j = 2, 0
        while i < m:
            # 如果 pattern[i-1] 和 pattern[j] 相同，那么 nxt[i] = 1 + nxt[i - 1]
            if pattern[i - 1] == pattern[j]:
                j += 1
                nxt[i] = j
                # i 位置的信息求完了接着求 i+1 位置的信息
                i += 1
            # 如果 pattern[i-1] 和 pattern[j] 不相等那么就往前跳
            elif j > 0:
                j = nxt[j]
            else:
                nxt[i] = 0
                i += 1
        return nxt
    n, m = len(text), len(pattern)
    if n == 0 or m < 1 or n < m:
        return - 1
    i, j = 0, 0
    nxt = construct(pattern, m)
    while i < n and j < m:
        if text[i] == pattern[j]:
            i += 1
            j += 1
        elif nxt[j] == -1:
            i += 1
        else:
            j = nxt[j]
    # j 越界表示匹配完成 i 越界表示 text 中没有 pattern 的子串
    return i - j if j == m else -1

In [4]:
def RK(text, pattern):
    HashMap = {}
    n, m = len(text), len(pattern)
    for i in range(n - m + 1):
        if HashMap.get(text[i: i + m]) == None:
            HashMap[text[i: i + m]] = i
    return HashMap.get(pattern) if HashMap.get(pattern) != None else -1

## Manacher 算法

1. 解决的问题

字符串 string 中，最长回文子串的长度如何求解？如何做到时间复杂度O(N)完成？

- 中心扩散法

以每个位置为中心向两边扩散

长度为偶数的回文可以通过添加特殊字符

例如：字符串```122131221```可以添加```#```得到```#1#2#2#1#3#1#2#2#1#```然后再使用中心扩散法即可，特殊字符无要求既可以在原字符串中出现过的也可以是没出现过的

- Manacher

**回文直径**：以某个中心向两边扩，括出整个区域的大小叫回文直径

**回文半径**：回文直径的一半再加 1 

例如：字符串```#a#1#2#1#b#```以 2 为中心向两边括，那么 2 的回文直径就是 7 回文半径就是 4

**回文半径数组**：从左往右遍历过程中将回文半径记录到一个数组中

**之前括的所有位置中所到达的最右回文右边界**：只要括的范围的右边界突破之前记录的值就更新

例如：字符串```#1#2#2#1#...```

| index | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | ... |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| character | # | 1 | # | 2 | # | 2 | # | 1 | # | ... |

一开始 $R=-1$ 表示开没开始括，然后以 $0$ 号位置为中心开始括得到的回文子串是 ```#``` 然后更新 $R=0$ 再以 $1$ 号位置为中心开始括得到的回文子串是```#1#``` 然后将更新 $R=2$

**取得更远边界中心点在哪**


几种情况：

1. 当来到某个中心点，这个点没有在最右回文右边界里：暴力括

例子：字符串```#1#2#1#...```


| index | 0 | 1 | 2 | 3 | 4 | 5 | 6 | ... |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| character | # | 1 | # | 2 | # | 1 | # | ... |

当来到 $0$ 号位置时由于一开始 $R=-1$ 所以需要暴力括

In [ ]:
def manacher(s):
    # 1221 -> #1#2#2#1#
    s1 = []
    for c in s:
        s1.append('#')
        s1.append(c)
        s1.append('#')
    r = -1
    c = -1
    for i in range(len(s)):
        # i 在 r 外部
        # 以 i 为中心往两边暴力括
        if i 